# Preprocess data into Vector DataBase

First step is to process data into vectordb, so that we can create a retirever to load the data.

### Load the data
There are many loaders being supported in langchain. I choose PDF loader here.

In [6]:
from langchain.document_loaders import PyMuPDFLoader
path="道路交通安全規則.pdf"
loader = PyMuPDFLoader(path)
pdf_data = loader.load()

# Use the RecursiveCharacterTextSplitter to split the text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=5)
all_splits = text_splitter.split_documents(pdf_data)

### Embedding Model

There are many embedding models, I choose two embedding models to conduct the experiment:
- `text-embedding-3-small`: Charged
- `all-MiniLM-L6-v2`: Free

In [7]:
embedding_name = "text-embedding-3-small"

if embedding_name == "text-embedding-3-small":
    from key import API_KEY
    from langchain.embeddings import OpenAIEmbeddings
    import os
    os.environ["OPENAI_API_KEY"] = API_KEY
    embedding_openai = OpenAIEmbeddings(model="text-embedding-3-small")
elif embedding_name == "all-MiniLM-L6-v2":
    from langchain.embeddings import HuggingFaceEmbeddings
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    model_kwargs = {'device': 'cpu'}
    embedding = HuggingFaceEmbeddings(model_name=model_name,
                                    model_kwargs=model_kwargs)
else:
    raise NotImplementedError

### Create the vector database

In [8]:
from langchain.vectorstores import Chroma
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_directory)

## Language Model
Here we use the api service provided by `llamafactory-cli` to hold our LLM.

Use the following command to setup the service:
```
API_PORT=8000 llamafactory-cli api model_info.yaml
```

After we've create the host, we can use `ChatOpenAI` to get the output.

To combine llm with langchain, the office suggests us to use [LCEL (LangChain Expression Language)](https://python.langchain.com/v0.1/docs/expression_language/) to formualate the chain.

In [22]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    openai_api_key='None', 
    openai_api_base='http://0.0.0.0:8000/v1',
    temperature=0.5,    
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Traditional Chinese. Translate the user sentence.",
    ),
    ("human", "Nice to meet you! What a wonderful day!"),
]

# LECL example
chain = (
    llm # The OpenAI model
    | StrOutputParser() # Parse the output of the OpenAI model
)

print(chain.invoke(messages))

Traditional Chinese translation:

好久不見！今天真好！


## Normal LLM versus RAG

First, is the chain of normal LLM.

We only apply template and output parser before and after the inference of llm.

In [23]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    openai_api_key='None', 
    openai_api_base='http://0.0.0.0:8000/v1',
    temperature=0
)

template = """Answer the following questions. If you don't know the answer, please say "I don't know".
---
Question {question}
"""

template = ChatPromptTemplate.from_template(template)

def log_prompt(prompt):
    # Print the prompt in green
    print("\033[92m{}\033[00m".format(prompt.to_string().replace("human: ", "")))  
    return prompt

normal_chain = (
    {"question": RunnablePassthrough()}
    | template
    | RunnableLambda(log_prompt)
    | llm
    | StrOutputParser()
)

Second, is the implementation of RAG.

Apart from what we've done, we also create a retriever when we get the user's input. 

Then we provide the infomation into the prompt, so that models can response better.

In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

retriever = vectordb.as_retriever()

llm = ChatOpenAI(
    openai_api_key='None', 
    openai_api_base='http://0.0.0.0:8000/v1',
    temperature=0
)

template = """Answer the following questions with the extra infomation below. If you don't know the answer, please say "I don't know".

Extra information:
```
{context}
```

---
Question {question}
"""

template = ChatPromptTemplate.from_template(template)

def log_prompt(prompt):
    # Print the prompt in green
    print("\033[92m{}\033[00m".format(prompt.to_string().replace("human: ", "")))  
    return prompt

def doc_format(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"question": RunnablePassthrough(), "context": retriever | doc_format} 
    | template
    | RunnableLambda(log_prompt)
    | llm
    | StrOutputParser()
)

Let's take a look of the result:

In [25]:
question = "請問大貨車載運劇團道具附載演員不得超過幾人呢？"
print(normal_chain.invoke(question))

Human: Answer the following questions. If you don't know the answer, please say "I don't know".
---
Question 請問大貨車載運劇團道具附載演員不得超過幾人呢？

I don't know


In [26]:
question = "請問大貨車載運劇團道具附載演員不得超過幾人呢？"
print(rag_chain.invoke(question))

Human: Answer the following questions with the extra infomation below. If you don't know the answer, please say "I don't know".

Extra information:
```
三、漁民攜帶大型捕魚工具，非客車所能容納者，搭載大貨車不得超過十六人，小貨車不得超過
八人。
四、大貨車載運劇團道具附載演員不得超過十六人，小貨車不得超過八人。

三、漁民攜帶大型捕魚工具，非客車所能容納者，搭載大貨車不得超過十六人，小貨車不得超過
八人。
四、大貨車載運劇團道具附載演員不得超過十六人，小貨車不得超過八人。

五、大貨車載運魚苗附載拍水人員不得超過十二人。
六、大貨車載運棺柩附載人員不得超過十六人。
七、大貨車載運神轎附載人員不得超過十六人。

五、大貨車載運魚苗附載拍水人員不得超過十二人。
六、大貨車載運棺柩附載人員不得超過十六人。
七、大貨車載運神轎附載人員不得超過十六人。
```

---
Question 請問大貨車載運劇團道具附載演員不得超過幾人呢？

根據提供的資訊，大貨車載運劇團道具附載演員不得超過十六人。
